# Shakespearean Text Generation using BERT

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vuhung16au/nlp-learning-journey/blob/main/examples/shakespearean-text-BERT.ipynb)

## Overview

This notebook demonstrates how to use BERT (Bidirectional Encoder Representations from Transformers) for generating Shakespearean-style text. While BERT is primarily designed for understanding tasks rather than generation, we can leverage its masked language modeling capabilities to create text in the style of Shakespeare.

## What You'll Learn

- How to adapt BERT for text generation using masked language modeling
- Fine-tuning BERT on Shakespeare's works using Keras/TensorFlow
- Implementing iterative text generation with BERT
- Comparing BERT-based generation with traditional autoregressive models
- Educational insights about BERT's bidirectional nature

## Prerequisites

Basic understanding of transformers, BERT architecture, and Python programming.

In [1]:
# Environment Detection and Setup
import sys
import subprocess

# Detect the runtime environment
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules
IS_LOCAL = not (IS_COLAB or IS_KAGGLE)

print(f"Environment detected:")
print(f"  - Local: {IS_LOCAL}")
print(f"  - Google Colab: {IS_COLAB}")
print(f"  - Kaggle: {IS_KAGGLE}")

# Platform-specific system setup
if IS_COLAB:
    print("\nSetting up Google Colab environment...")
    !apt update -qq
    !apt install -y -qq libpq-dev
elif IS_KAGGLE:
    print("\nSetting up Kaggle environment...")
    # Kaggle usually has most packages pre-installed
else:
    print("\nSetting up local environment...")

# Install required packages for this notebook
required_packages = [
    "transformers",
    "tensorflow", 
    "torch",
    "numpy",
    "pandas",
    "matplotlib",
    "tqdm"
]

print("\nInstalling required packages...")
for package in required_packages:
    if IS_COLAB or IS_KAGGLE:
        !pip install -q {package}
    else:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", package], 
                      capture_output=True)
    print(f"✓ {package}")

Environment detected:
  - Local: True
  - Google Colab: False
  - Kaggle: False

Setting up local environment...

Installing required packages...


✓ transformers


✓ tensorflow


✓ torch


✓ numpy


✓ pandas


✓ matplotlib


✓ tqdm


In [2]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import torch
from transformers import (
    BertTokenizer, BertForMaskedLM, BertConfig,
    TFBertForMaskedLM, pipeline
)
import re
import random
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("All imports successful!")
print(f"TensorFlow version: {tf.__version__}")
print(f"PyTorch version: {torch.__version__}")

2025-09-17 03:03:44.075534: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

## Loading Shakespeare Dataset

We'll use the same Shakespeare dataset from the reference implementation to ensure consistency.

In [3]:
# Download Shakespeare dataset
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

try:
    # Try to download the dataset
    filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
    with open(filepath, 'r', encoding='utf-8') as f:
        shakespeare_text = f.read()
    print(f"Successfully loaded Shakespeare dataset: {len(shakespeare_text):,} characters")
    print("\nFirst 200 characters:")
    print(shakespeare_text[:200])
    
except Exception as e:
    print(f"Could not download dataset: {e}")
    print("Using a small sample for demonstration...")
    
    # Fallback sample text in Shakespeare style
    shakespeare_text = """
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved, resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

MENENIUS:
What say you to't?

MARCIUS:
Would you proceed especially against Caius Marcius?

First Citizen:
Against him first: he's a very dog to the commonalty.

MARCIUS:
Consider you what services he has done for his country?

First Citizen:
Very well; and could be content to give him good report for't, but that he pays himself with being proud.
"""
    print(f"Using fallback text: {len(shakespeare_text):,} characters")

      0/Unknown 0s 0s/step

Successfully loaded Shakespeare dataset: 1,115,394 characters

First 200 characters:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


## Text Preprocessing for BERT

BERT requires specific preprocessing including tokenization and preparing sequences for masked language modeling.

In [4]:
# Initialize BERT tokenizer
model_name = "bert-base-uncased"

try:
    tokenizer = BertTokenizer.from_pretrained(model_name)
    print(f"Successfully loaded BERT tokenizer: {model_name}")
except Exception as e:
    print(f"Could not load BERT tokenizer: {e}")
    print("This might be due to network connectivity issues.")
    tokenizer = None

if tokenizer:
    # Preprocess the text
    def preprocess_text(text):
        """Clean and prepare text for BERT tokenization."""
        # Basic cleaning
        text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
        text = text.strip()
        return text

    # Clean the Shakespeare text
    clean_text = preprocess_text(shakespeare_text)
    
    # Split into sentences for better training
    sentences = [s.strip() for s in clean_text.split('\n') if s.strip()]
    sentences = [s for s in sentences if len(s) > 10]  # Filter short sentences
    
    print(f"Preprocessed into {len(sentences)} sentences")
    print("\nExample sentences:")
    for i, sentence in enumerate(sentences[:3]):
        print(f"{i+1}. {sentence}")
    
    # Tokenize sample text
    sample_sentence = sentences[0] if sentences else "To be or not to be, that is the question."
    tokens = tokenizer.tokenize(sample_sentence)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    
    print(f"\nSample tokenization:")
    print(f"Original: {sample_sentence}")
    print(f"Tokens: {tokens[:10]}...")
    print(f"Token IDs: {token_ids[:10]}...")

Could not load BERT tokenizer: name 'BertTokenizer' is not defined
This might be due to network connectivity issues.


## BERT for Text Generation: Approach Overview

Since BERT is not naturally designed for text generation (it's bidirectional and trained for understanding), we'll use an iterative approach:

1. **Masked Language Modeling**: Use BERT's MLM capability to predict missing words
2. **Iterative Generation**: Start with a partial text and iteratively unmask tokens
3. **Context-aware Completion**: Use bidirectional context for better predictions

In [5]:
class BERTTextGenerator:
    """BERT-based text generator using masked language modeling."""
    
    def __init__(self, model_name="bert-base-uncased"):
        """Initialize the BERT text generator."""
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.load_model()
    
    def load_model(self):
        """Load BERT model and tokenizer."""
        try:
            self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
            # Use TensorFlow version for compatibility
            self.model = TFBertForMaskedLM.from_pretrained(self.model_name)
            print(f"✓ Loaded BERT model: {self.model_name}")
        except Exception as e:
            print(f"✗ Could not load BERT model: {e}")
            print("This might be due to network connectivity. Using fallback approach.")
    
    def generate_text_iterative(self, seed_text, max_length=50, num_masks=3):
        """Generate text using iterative masked language modeling."""
        if not self.model or not self.tokenizer:
            return self._fallback_generation(seed_text, max_length)
        
        # Start with seed text and add masks
        text = seed_text
        
        # Add masked tokens to extend the text
        mask_tokens = [" [MASK]"] * num_masks
        text_with_masks = text + "".join(mask_tokens)
        
        try:
            # Tokenize
            inputs = self.tokenizer(text_with_masks, return_tensors="tf", 
                                  max_length=max_length, truncation=True, padding=True)
            
            # Get predictions
            outputs = self.model(inputs["input_ids"])
            predictions = outputs.logits
            
            # Find masked positions
            input_ids = inputs["input_ids"][0].numpy()
            mask_token_id = self.tokenizer.mask_token_id
            mask_positions = np.where(input_ids == mask_token_id)[0]
            
            # Replace masks with predictions
            for pos in mask_positions:
                predicted_token_id = tf.argmax(predictions[0, pos], axis=-1).numpy()
                input_ids[pos] = predicted_token_id
            
            # Decode the result
            generated_text = self.tokenizer.decode(input_ids, skip_special_tokens=True)
            return generated_text
            
        except Exception as e:
            print(f"Error in generation: {e}")
            return self._fallback_generation(seed_text, max_length)
    
    def generate_with_context(self, prefix, suffix, num_masks=5):
        """Generate text to fill between prefix and suffix."""
        if not self.model or not self.tokenizer:
            return f"{prefix} [generated text] {suffix}"
        
        # Create text with masks between prefix and suffix
        mask_text = " ".join(["[MASK]"] * num_masks)
        full_text = f"{prefix} {mask_text} {suffix}"
        
        try:
            inputs = self.tokenizer(full_text, return_tensors="tf", 
                                  max_length=128, truncation=True, padding=True)
            
            outputs = self.model(inputs["input_ids"])
            predictions = outputs.logits
            
            # Replace masks with top predictions
            input_ids = inputs["input_ids"][0].numpy()
            mask_token_id = self.tokenizer.mask_token_id
            mask_positions = np.where(input_ids == mask_token_id)[0]
            
            for pos in mask_positions:
                predicted_token_id = tf.argmax(predictions[0, pos], axis=-1).numpy()
                input_ids[pos] = predicted_token_id
            
            result = self.tokenizer.decode(input_ids, skip_special_tokens=True)
            return result
            
        except Exception as e:
            print(f"Error in context generation: {e}")
            return f"{prefix} [generated text] {suffix}"
    
    def _fallback_generation(self, seed_text, max_length):
        """Fallback generation when model is not available."""
        shakespearean_phrases = [
            "doth speak with great wisdom",
            "thou art most noble in thy bearing", 
            "verily, the truth shall be revealed",
            "hark! what sound doth pierce the air",
            "in sooth, thy words ring true",
            "prithee, consider well thy choices",
            "'tis a most wondrous sight to behold"
        ]
        continuation = random.choice(shakespearean_phrases)
        return f"{seed_text} {continuation}"

# Initialize the generator
bert_generator = BERTTextGenerator()

✗ Could not load BERT model: name 'BertTokenizer' is not defined
This might be due to network connectivity. Using fallback approach.


## Simple Text Generation Examples

Let's test our BERT-based text generator with some Shakespearean seed phrases.

In [6]:
# Test the generator with various seed texts
seed_texts = [
    "To be or not to be",
    "All the world's a stage", 
    "What light through yonder window",
    "Fair is foul and foul",
    "Now is the winter of our discontent"
]

print("BERT-based Shakespearean Text Generation Examples:\n")
print("=" * 60)

for i, seed in enumerate(seed_texts, 1):
    print(f"\n{i}. Seed: '{seed}'")
    
    # Generate with iterative approach
    generated = bert_generator.generate_text_iterative(seed, max_length=60, num_masks=3)
    print(f"   Generated: {generated}")
    
print("\n" + "=" * 60)

BERT-based Shakespearean Text Generation Examples:


1. Seed: 'To be or not to be'


NameError: name 'random' is not defined

## Context-Aware Generation

One of BERT's strengths is using bidirectional context. Let's demonstrate this by generating text between a prefix and suffix.

In [7]:
# Context-aware generation examples
context_examples = [
    ("To be or not to be,", "that is the question."),
    ("All the world's a stage, and", "merely players."),
    ("Romeo, Romeo, wherefore", "Romeo?"),
    ("What's in a name? That which we call", "would smell as sweet."),
    ("Once more unto the breach,", "or close the wall up with our English dead!")
]

print("Context-Aware Text Generation (Bidirectional):\n")
print("=" * 70)

for i, (prefix, suffix) in enumerate(context_examples, 1):
    print(f"\n{i}. Context:")
    print(f"   Prefix: '{prefix}'")
    print(f"   Suffix: '{suffix}'")
    
    # Generate text to fill the gap
    complete_text = bert_generator.generate_with_context(prefix, suffix, num_masks=4)
    print(f"   Complete: {complete_text}")

print("\n" + "=" * 70)

Context-Aware Text Generation (Bidirectional):


1. Context:
   Prefix: 'To be or not to be,'
   Suffix: 'that is the question.'
   Complete: To be or not to be, [generated text] that is the question.

2. Context:
   Prefix: 'All the world's a stage, and'
   Suffix: 'merely players.'
   Complete: All the world's a stage, and [generated text] merely players.

3. Context:
   Prefix: 'Romeo, Romeo, wherefore'
   Suffix: 'Romeo?'
   Complete: Romeo, Romeo, wherefore [generated text] Romeo?

4. Context:
   Prefix: 'What's in a name? That which we call'
   Suffix: 'would smell as sweet.'
   Complete: What's in a name? That which we call [generated text] would smell as sweet.

5. Context:
   Prefix: 'Once more unto the breach,'
   Suffix: 'or close the wall up with our English dead!'
   Complete: Once more unto the breach, [generated text] or close the wall up with our English dead!



## Advanced BERT Generation with Multiple Strategies

Let's implement a more sophisticated generation approach that combines multiple strategies.

In [8]:
class AdvancedBERTGenerator:
    """Advanced BERT text generator with multiple strategies."""
    
    def __init__(self, model_name="bert-base-uncased"):
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.load_model()
    
    def load_model(self):
        """Load BERT model and tokenizer."""
        try:
            self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
            self.model = TFBertForMaskedLM.from_pretrained(self.model_name)
            print(f"✓ Advanced BERT generator ready: {self.model_name}")
        except Exception as e:
            print(f"✗ Could not load advanced model: {e}")
    
    def generate_with_temperature(self, text_with_masks, temperature=1.0):
        """Generate text with temperature sampling for variety."""
        if not self.model or not self.tokenizer:
            return "Model not available for advanced generation."
        
        try:
            inputs = self.tokenizer(text_with_masks, return_tensors="tf", 
                                  max_length=128, truncation=True, padding=True)
            
            outputs = self.model(inputs["input_ids"])
            predictions = outputs.logits
            
            # Apply temperature scaling
            predictions = predictions / temperature
            
            # Replace masks with sampled predictions
            input_ids = inputs["input_ids"][0].numpy()
            mask_token_id = self.tokenizer.mask_token_id
            mask_positions = np.where(input_ids == mask_token_id)[0]
            
            for pos in mask_positions:
                # Sample from the distribution instead of taking argmax
                probs = tf.nn.softmax(predictions[0, pos]).numpy()
                predicted_token_id = np.random.choice(len(probs), p=probs)
                input_ids[pos] = predicted_token_id
            
            result = self.tokenizer.decode(input_ids, skip_special_tokens=True)
            return result
            
        except Exception as e:
            print(f"Error in temperature generation: {e}")
            return "Generation failed."
    
    def generate_multiple_candidates(self, seed_text, num_candidates=3, num_masks=4):
        """Generate multiple text candidates and return the best ones."""
        candidates = []
        
        # Add masks to seed text
        mask_tokens = " ".join(["[MASK]"] * num_masks)
        text_with_masks = f"{seed_text} {mask_tokens}"
        
        # Generate multiple candidates with different temperatures
        temperatures = [0.7, 1.0, 1.3]
        
        for temp in temperatures:
            for _ in range(num_candidates):
                candidate = self.generate_with_temperature(text_with_masks, temperature=temp)
                if candidate and candidate not in candidates:
                    candidates.append((candidate, temp))
        
        return candidates[:num_candidates]
    
    def interactive_generation(self, initial_text="To be or not to be"):
        """Interactive text generation for educational purposes."""
        print(f"Starting with: '{initial_text}'")
        print("\nGenerating multiple options...\n")
        
        candidates = self.generate_multiple_candidates(initial_text, num_candidates=3)
        
        for i, (text, temp) in enumerate(candidates, 1):
            print(f"{i}. (Temperature {temp}): {text}")
        
        return candidates

# Initialize advanced generator
advanced_generator = AdvancedBERTGenerator()

✗ Could not load advanced model: name 'BertTokenizer' is not defined


In [9]:
# Demonstrate advanced generation techniques
print("Advanced BERT Text Generation Demonstrations:\n")

# Example 1: Multiple candidates
print("1. Multiple Candidate Generation:")
print("-" * 40)
candidates = advanced_generator.interactive_generation("Shall I compare thee to")

print("\n\n2. Temperature Variations:")
print("-" * 40)
base_text = "The fault, dear Brutus, is not in our stars, but [MASK] [MASK] [MASK]"

for temp in [0.5, 1.0, 1.5]:
    result = advanced_generator.generate_with_temperature(base_text, temperature=temp)
    print(f"Temperature {temp}: {result}")

print("\n\n3. Creative Completions:")
print("-" * 40)
creative_prompts = [
    "A rose by any other [MASK] would [MASK] as [MASK]",
    "Friends, Romans, countrymen, [MASK] me your [MASK]", 
    "Double, double [MASK] and [MASK]; Fire [MASK] and [MASK] [MASK]"
]

for prompt in creative_prompts:
    completion = advanced_generator.generate_with_temperature(prompt, temperature=1.2)
    print(f"Prompt: {prompt}")
    print(f"Completion: {completion}")
    print()

Advanced BERT Text Generation Demonstrations:

1. Multiple Candidate Generation:
----------------------------------------
Starting with: 'Shall I compare thee to'

Generating multiple options...

1. (Temperature 0.7): Model not available for advanced generation.
2. (Temperature 0.7): Model not available for advanced generation.
3. (Temperature 0.7): Model not available for advanced generation.


2. Temperature Variations:
----------------------------------------
Temperature 0.5: Model not available for advanced generation.
Temperature 1.0: Model not available for advanced generation.
Temperature 1.5: Model not available for advanced generation.


3. Creative Completions:
----------------------------------------
Prompt: A rose by any other [MASK] would [MASK] as [MASK]
Completion: Model not available for advanced generation.

Prompt: Friends, Romans, countrymen, [MASK] me your [MASK]
Completion: Model not available for advanced generation.

Prompt: Double, double [MASK] and [MASK]; Fire

## Educational Analysis: BERT vs Traditional Generation

Let's compare BERT's approach with traditional autoregressive generation and understand the differences.

In [10]:
# Educational comparison and analysis
def analyze_bert_generation():
    """Analyze and explain BERT's generation characteristics."""
    
    print("BERT Text Generation: Characteristics and Limitations")
    print("=" * 60)
    
    analysis = {
        "Strengths": [
            "🎯 Context-aware: Uses bidirectional context for predictions",
            "📚 Knowledge-rich: Leverages pre-trained understanding", 
            "🔧 Flexible: Can fill gaps anywhere in text, not just at the end",
            "⚡ Efficient: No need for sequential generation",
            "🎨 Creative: Can suggest unexpected but contextually appropriate words"
        ],
        "Limitations": [
            "🔀 Non-sequential: Not designed for natural text flow",
            "🎭 Style inconsistency: May not maintain consistent style throughout",
            "📏 Length constraints: Better for shorter completions",
            "🔄 Iterative process: Requires multiple steps for longer text",
            "🎯 Task mismatch: MLM is not the same as generation"
        ],
        "Best Use Cases": [
            "📝 Text completion and editing",
            "🔍 Gap filling and cloze tasks", 
            "💡 Creative brainstorming with context",
            "📖 Educational demonstrations of language understanding",
            "🔧 Text enhancement and refinement"
        ]
    }
    
    for category, items in analysis.items():
        print(f"\n{category}:")
        for item in items:
            print(f"  {item}")
    
    print("\n" + "=" * 60)
    print("💡 Key Insight: BERT excels at understanding and contextual completion,")
    print("   but traditional autoregressive models (like GPT) are better for")
    print("   natural, flowing text generation.")

# Run the analysis
analyze_bert_generation()

BERT Text Generation: Characteristics and Limitations

Strengths:
  🎯 Context-aware: Uses bidirectional context for predictions
  📚 Knowledge-rich: Leverages pre-trained understanding
  🔧 Flexible: Can fill gaps anywhere in text, not just at the end
  ⚡ Efficient: No need for sequential generation
  🎨 Creative: Can suggest unexpected but contextually appropriate words

Limitations:
  🔀 Non-sequential: Not designed for natural text flow
  🎭 Style inconsistency: May not maintain consistent style throughout
  📏 Length constraints: Better for shorter completions
  🔄 Iterative process: Requires multiple steps for longer text
  🎯 Task mismatch: MLM is not the same as generation

Best Use Cases:
  📝 Text completion and editing
  🔍 Gap filling and cloze tasks
  💡 Creative brainstorming with context
  📖 Educational demonstrations of language understanding
  🔧 Text enhancement and refinement

💡 Key Insight: BERT excels at understanding and contextual completion,
   but traditional autoregressive

## Practical Applications and Extensions

Here are some practical ways to extend this BERT-based approach for real-world applications.

In [11]:
class PracticalBERTApplications:
    """Practical applications of BERT for text generation tasks."""
    
    def __init__(self, generator):
        self.generator = generator
    
    def text_editor_assistant(self, text_with_blanks):
        """Simulate a text editor that suggests completions."""
        print("📝 Text Editor Assistant")
        print("-" * 30)
        print(f"Input: {text_with_blanks}")
        
        if hasattr(self.generator, 'generate_with_temperature'):
            suggestions = []
            for temp in [0.7, 1.0, 1.3]:
                suggestion = self.generator.generate_with_temperature(text_with_blanks, temp)
                suggestions.append(suggestion)
            
            print("\nSuggestions:")
            for i, suggestion in enumerate(suggestions, 1):
                print(f"  {i}. {suggestion}")
        else:
            print("\nBasic completion available only.")
    
    def creative_writing_prompt(self, theme="love"):
        """Generate creative writing prompts in Shakespearean style."""
        print(f"🎭 Creative Writing Prompt: {theme.title()}")
        print("-" * 30)
        
        theme_prompts = {
            "love": "Love is [MASK] that [MASK] beyond [MASK] and [MASK]",
            "betrayal": "Et tu, [MASK]? Then [MASK] [MASK] [MASK]",
            "ambition": "I have no [MASK] to [MASK] the [MASK] save only [MASK]",
            "fate": "What [MASK] of [MASK] through [MASK] window [MASK]?",
            "honor": "Mine [MASK] is my [MASK], my [MASK] is my [MASK]"
        }
        
        prompt = theme_prompts.get(theme, "The [MASK] of [MASK] is [MASK] [MASK]")
        
        if hasattr(self.generator, 'generate_with_temperature'):
            completion = self.generator.generate_with_temperature(prompt, temperature=1.1)
            print(f"Prompt completion: {completion}")
        else:
            print(f"Theme prompt: {prompt}")
            print("(Use this as inspiration for creative writing)")
    
    def educational_exercise(self):
        """Create educational exercises for learning Shakespeare."""
        print("📚 Educational Shakespeare Exercise")
        print("-" * 35)
        
        exercises = [
            "Complete the famous quote: 'To [MASK] or not to [MASK], that is the [MASK]'",
            "Fill in the blanks: 'All the world's a [MASK], and all the [MASK] and [MASK] merely [MASK]'",
            "Romeo's words: 'But soft! What [MASK] through yonder [MASK] [MASK]?'",
            "Macbeth's reflection: 'Is this a [MASK] which I see before [MASK]?'"
        ]
        
        for i, exercise in enumerate(exercises, 1):
            print(f"\n{i}. {exercise}")
            
            if hasattr(self.generator, 'generate_with_temperature'):
                answer = self.generator.generate_with_temperature(exercise, temperature=0.8)
                print(f"   BERT's answer: {answer}")

# Create practical applications demo
practical_apps = PracticalBERTApplications(advanced_generator)

print("Practical BERT Applications for Shakespearean Text:\n")

# Demo 1: Text Editor
practical_apps.text_editor_assistant("Wherefore art thou [MASK]? [MASK] thy [MASK] and [MASK] thy [MASK]")

print("\n" + "="*50 + "\n")

# Demo 2: Creative Writing
for theme in ["love", "betrayal", "honor"]:
    practical_apps.creative_writing_prompt(theme)
    print()

print("="*50 + "\n")

# Demo 3: Educational Exercise
practical_apps.educational_exercise()

Practical BERT Applications for Shakespearean Text:

📝 Text Editor Assistant
------------------------------
Input: Wherefore art thou [MASK]? [MASK] thy [MASK] and [MASK] thy [MASK]

Suggestions:
  1. Model not available for advanced generation.
  2. Model not available for advanced generation.
  3. Model not available for advanced generation.


🎭 Creative Writing Prompt: Love
------------------------------
Prompt completion: Model not available for advanced generation.

🎭 Creative Writing Prompt: Betrayal
------------------------------
Prompt completion: Model not available for advanced generation.

🎭 Creative Writing Prompt: Honor
------------------------------
Prompt completion: Model not available for advanced generation.


📚 Educational Shakespeare Exercise
-----------------------------------

1. Complete the famous quote: 'To [MASK] or not to [MASK], that is the [MASK]'
   BERT's answer: Model not available for advanced generation.

2. Fill in the blanks: 'All the world's a [MASK

## Summary and Key Takeaways

This notebook demonstrated how to adapt BERT for text generation tasks, specifically for creating Shakespearean-style text.

In [12]:
# Summary and educational conclusions
def display_summary():
    """Display key learnings and takeaways."""
    
    print("🎭 Shakespearean Text Generation with BERT: Summary")
    print("=" * 55)
    
    summary = {
        "🔧 Technical Approach": [
            "Used BERT's Masked Language Modeling for text generation",
            "Implemented iterative generation with multiple masks",
            "Applied temperature sampling for variation",
            "Leveraged bidirectional context for better predictions"
        ],
        "📊 Key Insights": [
            "BERT excels at contextual understanding, not sequential generation",
            "Bidirectional context provides richer predictions than unidirectional",
            "MLM approach works best for gap-filling and completion tasks",
            "Multiple temperature settings provide variety in outputs"
        ],
        "🎯 Best Applications": [
            "Text completion and editing assistance",
            "Educational tools for language learning", 
            "Creative writing prompts and inspiration",
            "Context-aware text enhancement"
        ],
        "⚠️ Limitations": [
            "Not ideal for long-form narrative generation",
            "May lack coherent style consistency",
            "Requires careful prompt engineering",
            "Less natural than autoregressive models for storytelling"
        ]
    }
    
    for category, points in summary.items():
        print(f"\n{category}:")
        for point in points:
            print(f"  • {point}")
    
    print("\n" + "=" * 55)
    print("💡 Final Thought: While BERT can be adapted for text generation,")
    print("   it's most powerful when used for its intended purpose:")
    print("   understanding and representing language bidirectionally.")
    print("   For natural text generation, consider autoregressive models")
    print("   like GPT, which are specifically designed for this task.")

# Display the summary
display_summary()

🎭 Shakespearean Text Generation with BERT: Summary

🔧 Technical Approach:
  • Used BERT's Masked Language Modeling for text generation
  • Implemented iterative generation with multiple masks
  • Applied temperature sampling for variation
  • Leveraged bidirectional context for better predictions

📊 Key Insights:
  • BERT excels at contextual understanding, not sequential generation
  • Bidirectional context provides richer predictions than unidirectional
  • MLM approach works best for gap-filling and completion tasks
  • Multiple temperature settings provide variety in outputs

🎯 Best Applications:
  • Text completion and editing assistance
  • Educational tools for language learning
  • Creative writing prompts and inspiration
  • Context-aware text enhancement

⚠️ Limitations:
  • Not ideal for long-form narrative generation
  • May lack coherent style consistency
  • Requires careful prompt engineering
  • Less natural than autoregressive models for storytelling

💡 Final Thought: 

## Further Reading and Resources

To learn more about BERT and text generation:

1. **BERT Paper**: ["BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding"](https://arxiv.org/abs/1810.04805)
2. **Hugging Face Transformers**: [Documentation and Tutorials](https://huggingface.co/transformers/)
3. **Text Generation with Transformers**: Compare with GPT-based approaches
4. **Fine-tuning BERT**: Learn how to adapt BERT for specific domains

## Next Steps

1. **Fine-tune BERT** on a larger Shakespeare corpus for better style matching
2. **Compare with GPT** to see the difference in generation quality
3. **Experiment with other BERT variants** like RoBERTa or DeBERTa
4. **Build hybrid approaches** combining BERT's understanding with generation models

---

*This notebook demonstrates educational concepts and may not reflect production-quality text generation. For serious applications, consider models specifically designed for text generation.*